In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/newdata

/content/drive/MyDrive/newdata


In [ ]:
import sys
from pathlib import Path
from tqdm import tqdm
from pandas import HDFStore
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, RepeatVector




In [ ]:
directory_path = '/content/drive/MyDrive/newdata'
file_list = os.listdir(directory_path)
file_list

['mod01_out02_doe05_run001.akh5',
 'mod01_out02_doe05_run002.akh5',
 'mod01_out02_doe05_run003.akh5',
 'mod01_out02_doe05_run004.akh5',
 'mod01_out02_doe05_run005.akh5',
 'mod01_out02_doe05_run006.akh5',
 'mod01_out02_doe05_run007.akh5',
 'mod01_out02_doe05_run008.akh5',
 'mod01_out02_doe05_run009 (1).akh5',
 'mod01_out02_doe05_run010.akh5',
 'mod01_out02_doe05_run011.akh5',
 'mod01_out02_doe05_run012.akh5',
 'mod01_out02_doe05_run013.akh5',
 'mod01_out02_doe05_run014.akh5',
 'mod01_out02_doe05_run015.akh5',
 'mod01_out02_doe05_run016.akh5',
 'mod01_out02_doe05_run017.akh5',
 'mod01_out02_doe05_run018.akh5',
 'mod01_out02_doe05_run019.akh5',
 'mod01_out02_doe05_run020.akh5',
 'mod01_out02_doe05_run021.akh5',
 'mod01_out02_doe05_run022.akh5',
 'mod01_out02_doe05_run023.akh5',
 'mod01_out02_doe05_run024.akh5',
 'mod01_out02_doe05_run025.akh5',
 'mod01_out02_doe05_run026.akh5',
 'mod01_out02_doe05_run027.akh5',
 'mod01_out02_doe05_run028.akh5',
 'mod01_out02_doe05_run029.akh5',
 'mod01_ou

In [ ]:
def generate_data(obj):
    mass = obj.data['mass']
    shell_thickness = obj.data['SHELL_THICKNESS']
    data_y = obj.data['NODE_COOR_X']
    y_coordinates = data_y.iloc[:1, :]
    output_sequence = data_y.diff(axis=0).iloc[1:, :]
    return mass, shell_thickness, y_coordinates, output_sequence



In [ ]:
def preprocess_feature_engineering(mass, shell_thickness, y_coordinates, output_sequence):

    scaler_mass = StandardScaler()
    scaler_shell_thickness = StandardScaler()
    scaler_y_coordinates = StandardScaler()

    mass_scaled = scaler_mass.fit_transform(mass.values.reshape(-1, 1))
    shell_thickness_scaled = scaler_shell_thickness.fit_transform(shell_thickness.values.reshape(-1, 1))
    y_coordinates_scaled = scaler_y_coordinates.fit_transform(y_coordinates)


    scaler_minmax = MinMaxScaler()
    y_coordinates_minmax = scaler_minmax.fit_transform(y_coordinates)



    return mass_scaled, shell_thickness_scaled, y_coordinates_scaled, output_sequence


In [ ]:
from pathlib import Path
import shutil
import h5py
import numpy as np

import pandas as pd
from pandas import Series as Ds
from pandas import DataFrame as Df

import plotly.graph_objects as go
import plotly.express as px



def make_submission_folder(submission_template_folder=None, model_name=None):

    src = str(submission_template_folder)
    dst = Path(src).parent.joinpath(model_name)
    shutil.copytree(src, dst, dirs_exist_ok=True)
    return



class AkH5file():
    ''' AKH5 file is a hdf5 file containing node coordinates,
    element connectivities and attributes as function of time.
    This class provides methods to read and update coordinates of the file.
    '''
    _fig_height = 400
    _fig_width  = 400


    def __init__(self,file_path) -> None:
        self.root_name = 'All_Nodes'
        self.file_path = file_path
        self.file_name = Path(file_path).name

        self._open_file()
        self._read_all_datasets()
        self._close_file()


    def _open_file(self,mode='r'):
        self._h5file = h5py.File(self.file_path, mode=mode)
        self._root   = self._h5file[self.root_name]


    def _close_file(self):
        self._h5file.close()


    def _read_all_datasets(self):
        ''' Reading all data sets contained under All_Nodes group'''
        dataset_names = list(self._h5file[self.root_name].keys())
        self.data={}
        for name in dataset_names:
            self.data[name]=Df(self._root[name])


    @property
    def summary(self):
        summary = Ds(dtype=object)
        for name, dset in self.data.items():
            summary[name] = dset.shape
        return summary



    def __repr__(self) -> str:
        repr = self.summary.to_string()
        return repr


    def write_node_coords(self, coords_X=None , coords_Y=None, coords_Z=None):
        ''' Updates NODE_COOR_X Y and Z tables in ahh5 file.
        Inputs coords_x y z shape should be (n_time * n_nodes)
        '''
        self._open_file(mode='r+')
        # opts={'compression':'gzip','compression_opts':0}
        opts ={}

        del self._root['TIME']
        self._root.create_dataset(name='TIME',data=np.arange(0,82.5,2.5),**opts) # Writing in h5 file
        self.data['TIME']  = Df(np.arange(0,82.5,2.5)) # Just to keep data field in sync

        del self._root['NODE_COOR_X']
        self._root.create_dataset(name='NODE_COOR_X',data=coords_X,**opts) # Writing in h5 file
        self.data['NODE_COOR_X']  = Df(coords_X) # Just to keep data field in sync

        del self._root['NODE_COOR_Y']
        self._root.create_dataset(name='NODE_COOR_Y',data=coords_Y ,**opts) # Writing in h5 file
        self.data['NODE_COOR_Y']  = Df(coords_Y) # Just to keep data field in sync

        del self._root['NODE_COOR_Z']
        self._root.create_dataset(name='NODE_COOR_Z',data=coords_Z,**opts) # Writing in h5 file
        self.data['NODE_COOR_Z']  = Df(coords_Z) # Just to keep data field in sync

        self._close_file()


    def get_signal_rel_dist_nodes(self,node_id,node_id_ref,remove_offset=False):
        ''' Returns distance between nodes as function of time '''
        idx_nod = (self.data['DEFNODE']==node_id).values.ravel()
        idx_ref = (self.data['DEFNODE']==node_id_ref).values.ravel()

        X_nod = self.data['NODE_COOR_X'].values[:,idx_nod].ravel()
        Y_nod = self.data['NODE_COOR_Y'].values[:,idx_nod].ravel()
        Z_nod = self.data['NODE_COOR_Z'].values[:,idx_nod].ravel()

        X_ref = self.data['NODE_COOR_X'].values[:,idx_ref].ravel()
        Y_ref = self.data['NODE_COOR_Y'].values[:,idx_ref].ravel()
        Z_ref = self.data['NODE_COOR_Z'].values[:,idx_ref].ravel()

        X_vec = X_nod - X_ref
        Y_vec = Y_nod - Y_ref
        Z_vec = Z_nod - Z_ref
        res = (X_vec**2 + Y_vec**2 + Z_vec**2 )**.5
        if remove_offset:
            res = res - res[0]
        return res


    def plot3d(self,time=0):
        ''' Plots 3d node positions at a given time.
        The closest time is used for display
        '''
        # Display options
        opts_markers = dict(mode='markers', marker_symbol='square', marker_size=2, opacity=0.2, marker_color='green')
        opts_axis   = dict(showgrid=False, title='',showline=False,zeroline=False,ticks = '',
                           showticklabels = False, autorange=True)

        times_all    = self.data['TIME'].copy()
        times_all.columns = ['time']
        time_closest = self._filter_closest_df(times_all,'time',time)['time']
        time_index   = time_closest.index[0]

        x = self.data['NODE_COOR_X'].loc[time_index,:]
        y = self.data['NODE_COOR_Y'].loc[time_index,:]
        z = self.data['NODE_COOR_Z'].loc[time_index,:]

        # Mapping node id to index
        self.map_idx_uid_nodes = self.data['DEFNODE'].to_dict()[0]
        self.map_uid_idx_nodes = {v:k for k,v in self.map_idx_uid_nodes.items()}

        conn1 = self.data['SHELL2NODE'].iloc[:,[1,2,3]].applymap(lambda x: self.map_uid_idx_nodes[x])
        conn2 = self.data['SHELL2NODE'].iloc[:,[3,4,1]].applymap(lambda x: self.map_uid_idx_nodes[x])
        conn2.columns = conn1.columns
        conn = pd.concat([conn1,conn2])

        i= conn.iloc[:,0]
        j= conn.iloc[:,1]
        k= conn.iloc[:,2]

        geom = go.Mesh3d( x=x, y=y, z=z,  i=i, j=j, k=k,opacity=0.3)
        fig  = go.Figure(data=[geom])

        fig.update_scenes(aspectmode='data', xaxis=opts_axis, yaxis=opts_axis, zaxis=opts_axis)
        fig.update_layout({'scene':{"camera": {"projection":{"type": "orthographic"}}}})
                                                # {"type": "orthographic"}}}})
        fig.update_layout(height=self._fig_height, width=self._fig_width, margin=dict(l=0, r=0, t=35, b=0),
                          title=f'{self.file_name} : {time} ms')
        fig.update_layout(scene_camera=dict(eye=dict(x=2.5, y=0, z=0)))

        return fig


    def plot3danim(self,times=(0,200),skip=1):
        ''' Plots 3d animated mmesh.
        '''
        # Time frames to display
        time_values = self.data['TIME'].copy()
        time_values.columns=['time']
        time_values = time_values.query('@times[0] <= time <= @times[1]')['time'].values[::skip]

        # Spatial ranges
        x_range = [self.data['NODE_COOR_X'].min().min(),self.data['NODE_COOR_X'].max().max()]
        y_range = [self.data['NODE_COOR_Y'].min().min(),self.data['NODE_COOR_Y'].max().max()]
        z_range = [self.data['NODE_COOR_Z'].min().min(),self.data['NODE_COOR_Z'].max().max()]

        # Creating figure and frames
        fig  = go.Figure( data   = [self.plot3d(time=time_values[0]).data[0]],
                          frames = [go.Frame(data=[self.plot3d(time=t).data[0]],
                                    name=f'time{t:.2f}') for t in time_values])

        # Customise figure aspect
        opts_axis   = dict(showgrid=False, title='',showline=False,zeroline=False,ticks = '',
                           showticklabels = False, autorange=False)
        opts_axis_x = dict(range=x_range) | opts_axis
        opts_axis_y = dict(range=y_range) | opts_axis
        opts_axis_z = dict(range=z_range) | opts_axis

        fig.update_scenes(aspectmode='data', xaxis=opts_axis_x, yaxis=opts_axis_y, zaxis=opts_axis_z)
        fig.update_layout(height=self._fig_height, width=self._fig_width, margin=dict(l=0, r=0, t=35, b=0))
        fig.update_layout({'scene':{"camera": {"projection": {"type": "orthographic"}}}})
        fig.update_layout(scene_camera=dict(eye=dict(x=2.5, y=0, z=0)))

        # Create animation control
        fig.update_layout(updatemenus=[dict(type="buttons",
            buttons=[dict(label="Play",method="animate",
                    args=[None, {"frame": {"duration": 1, "redraw": True},
                    "fromcurrent": True, "transition": {"duration": 0}}])])])

        return fig




    # Helper functions --------------------------

    @staticmethod
    def _filter_closest_df(df,filtering_column,target_value):
        ''' Helper function to select dataframe rows where the filtering_column value is the closest to the target value'''
        closest_value = df.iloc[(df[filtering_column]-target_value).abs().argsort()[:1]][filtering_column].values[0]
        df_selection  = df[df[filtering_column]==closest_value]
        return df_selection


In [ ]:
for afile in file_list:
    print(afile)

mod01_out02_doe05_run001.akh5
mod01_out02_doe05_run002.akh5
mod01_out02_doe05_run003.akh5
mod01_out02_doe05_run004.akh5
mod01_out02_doe05_run005.akh5
mod01_out02_doe05_run006.akh5
mod01_out02_doe05_run007.akh5
mod01_out02_doe05_run008.akh5
mod01_out02_doe05_run009 (1).akh5
mod01_out02_doe05_run010.akh5
mod01_out02_doe05_run011.akh5
mod01_out02_doe05_run012.akh5
mod01_out02_doe05_run013.akh5
mod01_out02_doe05_run014.akh5
mod01_out02_doe05_run015.akh5
mod01_out02_doe05_run016.akh5
mod01_out02_doe05_run017.akh5
mod01_out02_doe05_run018.akh5
mod01_out02_doe05_run019.akh5
mod01_out02_doe05_run020.akh5
mod01_out02_doe05_run021.akh5
mod01_out02_doe05_run022.akh5
mod01_out02_doe05_run023.akh5
mod01_out02_doe05_run024.akh5
mod01_out02_doe05_run025.akh5
mod01_out02_doe05_run026.akh5
mod01_out02_doe05_run027.akh5
mod01_out02_doe05_run028.akh5
mod01_out02_doe05_run029.akh5
mod01_out02_doe05_run030.akh5
mod01_out02_doe05_run031.akh5
mod01_out02_doe05_run032.akh5
mod01_out02_doe05_run033.akh5
mod01_

In [ ]:
import numpy as np

def pad_array(arr, target_length):
    if len(arr) < target_length:
        return np.pad(arr, (0, target_length - len(arr)), constant_values=0)
    elif len(arr) > target_length:
        return arr[:target_length]
    else:
        return arr

processed_dataframes = []

for afile in tqdm(file_list, desc="Processing Datasets"):
    obj = AkH5file(afile)
    mass, shell_thickness, y_coordinates, output_sequence = generate_data(obj)


    mass_scaled, shell_thickness_scaled, y_coordinates_scaled, output_sequence = \
        preprocess_feature_engineering(mass, shell_thickness, y_coordinates, output_sequence)


    max_length = max(len(arr) for arr in [mass_scaled, shell_thickness_scaled, y_coordinates_scaled, output_sequence.values])


    mass_scaled = pad_array(mass_scaled.flatten(), max_length)
    shell_thickness_scaled = pad_array(shell_thickness_scaled.flatten(), max_length)
    y_coordinates_scaled = pad_array(y_coordinates_scaled.flatten(), max_length)
    output_sequence = pad_array(output_sequence.values.flatten(), max_length)


    df = pd.DataFrame({
        'mass_scaled': mass_scaled,
        'shell_thickness_scaled': shell_thickness_scaled,
        'y_coordinates_scaled': y_coordinates_scaled,
        'output_sequence': output_sequence,

    })

    processed_dataframes.append(df)


    print(len(mass_scaled), len(shell_thickness_scaled), len(y_coordinates_scaled), len(output_sequence))


Processing Datasets:   2%|▏         | 1/60 [00:01<01:22,  1.40s/it]

15718 15718 15718 15718


Processing Datasets:   3%|▎         | 2/60 [00:05<02:49,  2.92s/it]

15718 15718 15718 15718


Processing Datasets:   5%|▌         | 3/60 [00:06<02:06,  2.23s/it]

...

15718 15718 15718 15718


Processing Datasets:  98%|█████████▊| 59/60 [01:48<00:01,  1.40s/it]

15718 15718 15718 15718


Processing Datasets: 100%|██████████| 60/60 [01:49<00:00,  1.83s/it]

15718 15718 15718 15718


In [ ]:
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
from tensorflow.keras import backend as K

# Define or import processed_dataframes, X_train_list, y_train_list, max_length

# Reshape and pad the data
X_train_padded = []
max_length = max(len(df) for df in processed_dataframes)

for df in processed_dataframes:
    df_padded = df.copy()
    for col in df.columns:
        df_padded[col] = np.pad(df[col].values, (0, max_length - len(df[col])), constant_values=0)

    X_train_padded.append(df_padded[['mass_scaled', 'shell_thickness_scaled', 'y_coordinates_scaled']])

X_train_padded = np.array([X.values for X in X_train_padded])

# Define the number of epochs and batch size
epochs = 100  # Change the number of epochs as needed
batch_size = 10

# Generator
latent_dim = 100
generator_input = Input(shape=(latent_dim,))
x = Dense(128, activation='relu')(generator_input)
x = Dense(256, activation='relu')(x)
generator_output = Dense(max_length * X_train_padded.shape[2], activation='linear')(x)
generator_output = Reshape((max_length, X_train_padded.shape[2]))(generator_output)
generator = Model(generator_input, generator_output)

# Discriminator
discriminator_input = Input(shape=(max_length, X_train_padded.shape[2]))
x = Flatten()(discriminator_input)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
discriminator_output = Dense(1, activation='sigmoid')(x)
discriminator = Model(discriminator_input, discriminator_output)
discriminator.compile(optimizer=Adam(learning_rate=0.0002), loss='mse')  # Change loss to 'mse'

# Combined Model (GAN)
gan_input = Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = Model(gan_input, gan_output)
gan.compile(optimizer=Adam(learning_rate=0.0002), loss='mse')  # Change loss to 'mse'

# Training Loop
for epoch in range(epochs):
    print(f'Epoch {epoch + 1}/{epochs}:')
    for i in range(0, len(X_train_padded), batch_size):
        X_train_batch = X_train_padded[i:i+batch_size]

        # Sample random noise as input to the generator
        noise = np.random.normal(0, 1, (len(X_train_batch), latent_dim))

        # Generate a batch of new sequences
        generated_sequences = generator.predict(noise)

        # Reshape X_train_batch to match the shape of generated_sequences
        X_train_batch_reshaped = X_train_batch.reshape(-1, max_length, X_train_padded.shape[2])

        # Combine generated sequences with real sequences
        X_combined = np.concatenate([X_train_batch_reshaped, generated_sequences])
        y_combined = np.concatenate([np.ones((len(X_train_batch_reshaped), 1)), np.zeros((len(generated_sequences), 1))])

        # Train discriminator
        d_loss = discriminator.train_on_batch(X_combined, y_combined)

        # Train generator (via GAN)
        noise = np.random.normal(0, 1, (len(X_train_batch_reshaped), latent_dim))
        y_gan = np.ones((len(X_train_batch_reshaped), 1))
        g_loss = gan.train_on_batch(noise, y_gan)

    # Print progress after each epoch
    print(f'Discriminator Loss: {d_loss}, Generator Loss: {g_loss}')

    # Clear session to release memory
    K.clear_session()


Epoch 1/100:
1/1 [==============================] - 0s 34ms/step
Discriminator Loss: 0.23483283817768097, Generator Loss: 0.14188380539417267
Epoch 2/100:
1/1 [==============================] - 0s 50ms/step
Discriminator Loss: 0.36599424481391907, Generator Loss: 0.057647954672575
Epoch 3/100:
1/1 [==============================] - 0s 98ms/step
Discriminator Loss: 0.420909583568573, Generator Loss: 0.022900324314832687
Epoch 4/100:
1/1 [==============================] - 0s 32ms/step
Discriminator Loss: 0.4410712718963623, Generator Loss: 0.021293440833687782
Epoch 5/100:
1/1 [==============================] - 0s 35ms/step
Discriminator Loss: 0.43669548630714417, Generator Loss: 0.02055438607931137
Epoch 6/100:
1/1 [==============================] - 0s 33ms/step
Discriminator Loss: 0.44442039728164673, Generator Loss: 0.01544285099953413
Epoch 7/100:
1/1 [==============================] - 0s 32ms/step
Discriminator Loss: 0.4666374623775482, Generator Loss: 0.012909254059195518
Epoch 8/1

In [ ]:
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.losses import mse
import numpy as np
import tensorflow as tf


# Define or import processed_dataframes, X_train_list, y_train_list, max_length
max_length = max(len(df) for df in processed_dataframes)

# Reshape and pad the data
X_train_padded = []
for df in processed_dataframes:
    df_padded = df.copy()
    for col in df.columns:
        df_padded[col] = np.pad(df[col].values, (0, max_length - len(df[col])), constant_values=0)
    X_train_padded.append(df_padded[['mass_scaled', 'shell_thickness_scaled', 'y_coordinates_scaled']])

X_train_padded = np.array([X.values for X in X_train_padded])

# Define the number of epochs and batch size
epochs = 100  # Change the number of epochs as needed
batch_size = 10

# Define latent dimension
latent_dim = 100

# Encoder
encoder_input = Input(shape=(max_length, X_train_padded.shape[2]))
x = Flatten()(encoder_input)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
z_mean = Dense(latent_dim)(x)
z_log_var = Dense(latent_dim)(x)

# Sampling function
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0., stddev=1.)
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# Decoder
decoder_input = Input(shape=(latent_dim,))
x = Dense(256, activation='relu')(decoder_input)
x = Dense(512, activation='relu')(x)
decoder_output = Dense(max_length * X_train_padded.shape[2], activation='linear')(x)
decoder_output = Reshape((max_length, X_train_padded.shape[2]))(decoder_output)

# Instantiate Encoder and Decoder models
encoder = Model(encoder_input, [z_mean, z_log_var, z])
decoder = Model(decoder_input, decoder_output)

# VAE Model
vae_output = decoder(z)
vae = Model(encoder_input, vae_output)

# Define VAE loss
def vae_loss(x, x_decoded_mean, z_log_var=z_log_var, z_mean=z_mean):
    reconstruction_loss = mse(K.flatten(x), K.flatten(x_decoded_mean))
    kl_loss = -0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    return reconstruction_loss + kl_loss

# Compile VAE
vae.compile(optimizer=Adam(learning_rate=0.0002), loss=vae_loss)

# Custom training loop
for epoch in range(epochs):
    print(f'Epoch {epoch + 1}/{epochs}:')
    for df_idx in range(len(processed_dataframes)):
        X_train_batch = np.expand_dims(X_train_padded[df_idx], axis=0)

        # Perform a forward pass
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = encoder(X_train_batch)
            x_decoded = decoder(z)

            # Compute loss
            loss = vae_loss(X_train_batch, x_decoded, z_log_var, z_mean)

        # Compute gradients
        grads = tape.gradient(loss, vae.trainable_weights)

        # Update weights
        vae.optimizer.apply_gradients(zip(grads, vae.trainable_weights))

    # Print progress after each epoch
    print(f'Epoch {epoch + 1}/{epochs}, VAE Loss: {loss.numpy()}')


Epoch 1/100:
Epoch 1/100, VAE Loss: [0.03524687]
Epoch 2/100:
Epoch 2/100, VAE Loss: [0.00602882]
Epoch 3/100:
Epoch 3/100, VAE Loss: [0.01462363]
Epoch 4/100:
Epoch 4/100, VAE Loss: [0.00159044]
Epoch 5/100:
Epoch 5/100, VAE Loss: [0.0028776]
Epoch 6/100:
Epoch 6/100, VAE Loss: [0.01199972]
Epoch 7/100:
Epoch 7/100, VAE Loss: [0.02823019]
Epoch 8/100:
Epoch 8/100, VAE Loss: [0.00535634]
Epoch 9/100:
Epoch 9/100, VAE Loss: [0.00785968]
Epoch 10/100:
Epoch 10/100, VAE Loss: [0.01198674]
Epoch 11/100:
Epoch 11/100, VAE Loss: [0.00145101]
Epoch 12/100:
Epoch 12/100, VAE Loss: [0.00356273]
Epoch 13/100:
Epoch 13/100, VAE Loss: [0.00308148]
Epoch 14/100:
Epoch 14/100, VAE Loss: [0.00207167]
Epoch 15/100:
Epoch 15/100, VAE Loss: [0.00515618]
Epoch 16/100:
Epoch 16/100, VAE Loss: [0.00114416]
Epoch 17/100:
Epoch 17/100, VAE Loss: [0.01155172]
Epoch 18/100:
Epoch 18/100, VAE Loss: [0.00642517]
Epoch 19/100:
Epoch 19/100, VAE Loss: [0.00474873]
Epoch 20/100:
Epoch 20/100, VAE Loss: [0.00304886]